In [203]:
import pandas as pd 
import numpy as np
import torch
from torch import nn

import torch.nn.functional as F
from torchvision import datasets, transforms


In [204]:
print("I am here!")

my_data = pd.read_csv("train.csv", index_col = 0)

my_data = my_data.drop(columns = ['batch_size_test', 'batch_size_val', 'criterion', 'optimizer', 'batch_size_train'])
y_train = my_data['train_error']
y_val = my_data['val_error']
my_data = my_data.drop(columns = 'train_error')
my_data = my_data.drop(columns = 'val_error')

x = my_data.drop(columns = ['train_loss','val_loss','arch_and_hp','init_params_mu', 'init_params_std', 'init_params_l2'])

col_train = [col for col in x.filter(regex='^train',axis=1).columns]
col_val = [col for col in x.filter(regex='^val',axis=1).columns]
#print(col_train)
#print(col_val)
#print(x)

torch_tensor_train = torch.tensor([x[title].values for title in col_train],dtype=torch.float)
torch_tensor_val = torch.tensor([x[title].values for title in col_val],dtype=torch.float)


torch_tensor_train = torch.t(torch_tensor_train)
torch_tensor_val = torch.t(torch_tensor_val)


print(len(col_train))
print(torch_tensor_train.size())

# ------------- test part -----------------------

test_data = pd.read_csv("test.csv", index_col = 0)
test_data = test_data.drop(columns = ['batch_size_test', 'batch_size_val', 'criterion', 'optimizer', 'batch_size_train'])
test_data = test_data.drop(columns = ['arch_and_hp','init_params_mu', 'init_params_std', 'init_params_l2'])
test_train = test_data[col_train]
test_val = test_data[col_val]

torch_tensor_test_train = torch.tensor([test_train[title].values for title in col_train],dtype=torch.float)
torch_tensor_test_val = torch.tensor([test_val[title].values for title in col_val],dtype=torch.float)

torch_tensor_test_train = torch.t(torch_tensor_test_train)
torch_tensor_test_val = torch.t(torch_tensor_test_val)

#print(test_train)
print(torch_tensor_test_train.size())

I am here!
100
torch.Size([1878, 100])
torch.Size([476, 100])


In [211]:
#we first select a naive model after sorting based on the lowest val_error

#model = torch.nn.Sequential(nn.BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), nn.Conv2d(3, 32, kernel_size=(1, 1), stride=(1, 1)) , nn.Tanh() , nn.Dropout2d(p=0.4569261378573387, inplace=False) , nn.Conv2d(32, 27, kernel_size=(3, 3), stride=(1, 1)) , nn.ReLU() , nn.Dropout2d(p=0.022897457910152297, inplace=False) , nn.Conv2d(27, 28, kernel_size=(5, 5), stride=(1, 1)) , nn.LeakyReLU(negative_slope=0.5719652455913541) , nn.MaxPool2d(kernel_size=5, stride=5, padding=0, dilation=1, ceil_mode=False) , nn.Flatten() , nn.Linear(in_features=700, out_features=28, bias=True) , nn.LeakyReLU(negative_slope=0.48187737007312437) , nn.Dropout(p=0.3735548245474202, inplace=False) , nn.Linear(in_features=28, out_features=10, bias=True) , nn.BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True) )


input_size = 100
hidden_sizes = [100, 100]
output_size = 1

class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size()[0], -1)


model_train = nn.Sequential(
                      nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[1], output_size),
                      )
#nn.Softmax(dim=0)
# Define the loss
criterion = nn.MSELoss()
# Optimizers require the parameters to optimize and a learning rate
optimizer = torch.optim.SGD(model_train.parameters(), lr=0.03)
epochs = 100
for e in range(epochs):
    running_loss = 0
    for i in range(1878):
        # Training pass
        optimizer.zero_grad()
        
        output = model_train(torch_tensor_train[i])
        #print(output)
        #torch.tensor(y_train[i])
        loss = criterion(output, torch.tensor(y_train[i]))
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        
model_val = nn.Sequential(
                      nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[1], output_size),
                      )

# Optimizers require the parameters to optimize and a learning rate
optimizer = torch.optim.SGD(model_val.parameters(), lr=0.03)
epochs = 100
for e in range(epochs):
    running_loss = 0
    for i in range(1878):
        # Training pass
        optimizer.zero_grad()
        
        output = model_val(torch_tensor_val[i])
        #print(output)
        loss = criterion(output, torch.tensor(y_val[i]))
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

print("finished")



/Users/Wusongfeng/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


finished


In [221]:

#now we do the prediction

print("I am here!")
res_train,res_val = [],[]

for row in torch_tensor_test_train:
    res_train.append(model_train(row).item())
    
for row in torch_tensor_test_val:
    res_val.append(model_val(row).item())

print(res_val)
print(res_train)

I am here!
[0.5583013296127319, 0.4050866365432739, 0.5262130498886108, 0.680777907371521, 0.5923596024513245, 0.5499338507652283, 0.7253270745277405, 0.42069774866104126, 0.6739445328712463, 0.31737637519836426, 0.46779578924179077, 0.3898645043373108, 0.36771681904792786, 0.49539193511009216, 0.4169318974018097, 0.5172078609466553, 0.4393386244773865, 0.43747478723526, 0.38684356212615967, 0.4562339782714844, 0.3852398097515106, 0.4175161123275757, 0.6088973879814148, 0.28341975808143616, 0.5611382126808167, 0.47171351313591003, 0.6685824394226074, 0.3603111207485199, 0.45174649357795715, 0.3378792703151703, 0.7338246703147888, 0.38442370295524597, 0.6988584399223328, 0.4749460518360138, 0.4936230480670929, 0.6209245324134827, 0.5204269886016846, 0.625515878200531, 0.46746188402175903, 0.45091986656188965, 0.6439609527587891, 0.6431218981742859, 0.7168214917182922, 0.6156345009803772, 0.45250609517097473, 0.38995665311813354, 0.5342491269111633, 0.44317930936813354, 0.513914763927459

In [222]:
submit = pd.read_csv("sample_submission.csv", index_col = 0)
res_array = np.zeros(len(res_train)*2)
submit.Predicted = res_array

i = 0
j = 0
while True:
    
    if i == len(res_train):
        break
    
    res_array[j] = res_val[i]
    j += 1
    res_array[j] = res_train[i]
    j += 1
    
    i += 1 
    
submit.to_csv("submission.csv")